In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import spotipy
import sys
import pprint
import warnings
warnings.filterwarnings("ignore")
import math
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id='653cfed9a26e46dca3e3e211ded03c0f',\
                                                      client_secret='a14b7886714c41c383458b26434d60cc')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
import spotipy
import json
import time
import copy
import scipy.sparse as ssp
from collections import Counter, defaultdict
import tqdm
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
from scipy.stats import pearsonr
from __future__ import print_function    
import time
from scipy import stats as sss
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.neural_network import MLPRegressor
import pickle
from scipy import io
from sklearn.cluster import KMeans

/home/ubuntu/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
validation_df = pd.read_csv('validation_df.csv')

In [3]:
validation_df = validation_df.drop('Unnamed: 0',axis=1)

# Generate user listen histroy by dictionary

In [4]:
music = validation_df[['customer_id','track_name']]
music.sort_values('customer_id',inplace=True)

# track_set = music['track_name'].unique() ######
users_, tracks = music.T.values

In [5]:
user, index = np.unique(users_,True)
user_history = np.split(tracks,index[1:])

# index_to_user_dict = dict(enumerate(user,start=0))
# index_to_track_id = dict(enumerate(track_set,start=0))

# inv_track_id_map = {v: k for k, v in index_to_track_id.items()}

In [6]:
v={u:defaultdict(int) for u in user}

for u,history in zip(user,user_history):
        for song in history:
            v[u][song]+=1

In [7]:
t = np.load('trainset_Dict.npy').item()
index_to_user_dict = np.load('train_user_map.npy').item()
index_to_track_id = np.load('train_track_map.npy').item()
inv_track_id_map = np.load('inv_track_map.npy').item()

In [8]:
v_user = v.keys()
v_f = {}
non_ative = []
n=0

for i in tqdm.tqdm(range(15000)):
    if index_to_user_dict[i] in v_user:
        v_f[index_to_user_dict[i]] = v[index_to_user_dict[i]]
        
    else:
        n+=1
        non_ative.append(i)
        v_f[index_to_user_dict[i]] = t[index_to_user_dict[i]]

100%|██████████| 15000/15000 [00:00<00:00, 303044.97it/s]


In [9]:
n

3289

In [10]:
track_base = []
for i in tqdm.tqdm(range(15000)):
    track_base.extend(list(t[index_to_user_dict[i]].keys()))
track_base = set(track_base)

col_num = len(track_base)

100%|██████████| 15000/15000 [00:00<00:00, 256237.72it/s]


In [11]:
val_mat = ssp.dok_matrix((15000,col_num))

for i in tqdm.tqdm(range(15000)):
    for j in v_f[index_to_user_dict[i]].keys():
        if j in track_base:
            k = inv_track_id_map[j]
            val_mat[i,k] = v_f[index_to_user_dict[i]][index_to_track_id[k]]

100%|██████████| 15000/15000 [00:00<00:00, 26237.55it/s]


In [21]:
io.mmwrite("val_mat_17.mtx", val_mat)
# val_mat = io.mmread("val_mat_17.mtx")
col_num=19977

### Evaluate 1: MAE and RMSE of predict with ground truth

In [18]:
user_cosine = io.mmread("user_cosine_17.mtx")
user_item = io.mmread("user_item.mtx")
user_cosine = user_cosine.tocsr()
user_item = user_item.tocsr()

In [19]:
mat_col = np.array(range(col_num))

In [20]:
val_mat = val_mat.tocsr()

In [22]:
### Calculate the MAE and RMSE with user-based cosine similarity
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    minus = user_cosine[i,pred_col].toarray() - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE_1 = MAE_numerator/value_denominator
RMSE_1 =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of user-based cosine similarity is',MAE_1)
print('')
print('RMSE of user-based cosine similarity is',RMSE_1)

100%|██████████| 15000/15000 [00:25<00:00, 589.04it/s]

MAE of user-based cosine similarity is 0.0381353886374

RMSE of user-based cosine similarity is 0.6516424533543341


In [24]:
user_Pearson = io.mmread("user_Pearson_17.mtx")
user_Pearson = user_Pearson.tocsr()

In [10]:
# user_Pearson.todense()

In [25]:
### Calculate the MAE and RMSE with user-based Pearson correlation
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    minus = user_Pearson[i,pred_col].toarray() - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE_2 = MAE_numerator/value_denominator
RMSE_2 =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of user-based Pearson Correlation is',MAE_2)
print('')
print('RMSE of user-based Pearson Correlation is',RMSE_2)

100%|██████████| 15000/15000 [00:24<00:00, 604.12it/s]

MAE of user-based Pearson Correlation is 0.00757784929064

RMSE of user-based Pearson Correlation is 0.0888499763479117


In [26]:
Item_cosine = io.mmread("Item_cosine_17.mtx")
Item_cosine = Item_cosine.tocsr()

In [27]:
### Calculate the MAE and RMSE with item-based cosine similarity
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    minus = Item_cosine[i,pred_col].toarray() - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE_3 = MAE_numerator/value_denominator
RMSE_3 =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of Item cosine similarity is',MAE_3)
print('')
print('RMSE of Item cosine similarity is',RMSE_3)

100%|██████████| 15000/15000 [00:25<00:00, 590.66it/s]

MAE of Item cosine similarity is 0.00611857296165

RMSE of Item cosine similarity is 0.0497896665879327


In [28]:
Item_adjusted_cosine = io.mmread("Item_adjusted_cosine_17.mtx")
Item_adjusted_cosine = Item_adjusted_cosine.tocsr()

In [29]:
### Calculate the MAE and RMSE with item-based adjusted cosine similarity
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    minus = Item_adjusted_cosine[i,pred_col].toarray() - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE_4 = MAE_numerator/value_denominator
RMSE_4 =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of Item adjusted cosine similarity is',MAE_4)
print('')
print('RMSE of Item adjusted cosine similarity is',RMSE_4)

100%|██████████| 15000/15000 [00:24<00:00, 601.91it/s]

MAE of Item adjusted cosine similarity is 0.00511745425269

RMSE of Item adjusted cosine similarity is 0.053856400199402854


In [31]:
Item_Pearson = io.mmread("Item_Pearson_17.mtx")
Item_Pearson = Item_Pearson.tocsr()

In [32]:
### Calculate the MAE and RMSE with item-based Pearson Correlation
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    minus = Item_Pearson[i,pred_col].toarray() - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE_5 = MAE_numerator/value_denominator
RMSE_5 =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of Item Pearson Correlation is',MAE_5)
print('')
print('RMSE of Item Pearson Correlation is',RMSE_5)

100%|██████████| 15000/15000 [00:24<00:00, 612.62it/s]

MAE of Item Pearson Correlation is 0.00208821904156

RMSE of Item Pearson Correlation is 0.035055190045670835


In [33]:
### Calculate the MAE and RMSE with average of all similarity based results
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    avg_pred = (user_cosine[i,pred_col].toarray()+user_Pearson[i,pred_col].toarray()+\
               Item_cosine[i,pred_col].toarray()+Item_adjusted_cosine[i,pred_col].toarray()+
               Item_Pearson[i,pred_col].toarray())/5
    minus = avg_pred - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE = MAE_numerator/value_denominator
RMSE =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of average of five similarity pred is',MAE)
print('')
print('RMSE of average of five similarity pred is',RMSE)

100%|██████████| 15000/15000 [01:10<00:00, 213.27it/s]

MAE of average of five similarity pred is 0.00926240764814

RMSE of average of five similarity pred is 0.1393824411039587


In [34]:
Content_based = io.mmread("Content_based_17.mtx")
Content_based = Content_based.tocsr()

In [35]:
### Calculate the MAE and RMSE with content-based algorithm
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    minus = Content_based[i,pred_col].toarray() - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE_6 = MAE_numerator/value_denominator
RMSE_6 =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of Content based cosine is',MAE_6)
print('')
print('RMSE of Content based cosine is',RMSE_6)

100%|██████████| 15000/15000 [00:32<00:00, 467.81it/s]

MAE of Content based cosine is 6.42139079337

RMSE of Content based cosine is 11.874824639006192


In [36]:
### Calculate the MAE and RMSE with average of all similarity based results
value_denominator = 0
MAE_numerator = 0
RMSE_numerator = 0
for i in tqdm.tqdm(range(15000)):
    del_col = user_item[i,:].indices
    pred_col = np.delete(mat_col,del_col)
    avg_pred = (user_cosine[i,pred_col].toarray()+user_Pearson[i,pred_col].toarray()+\
               Item_cosine[i,pred_col].toarray()+Item_adjusted_cosine[i,pred_col].toarray()+
               Item_Pearson[i,pred_col].toarray()+Content_based[i,pred_col].toarray())/6
    minus = avg_pred - val_mat[i,pred_col].toarray()
    one_MAE = np.sum(np.absolute(minus))
    one_RMSE = np.sum(minus**2)
    MAE_numerator = MAE_numerator + one_MAE
    RMSE_numerator = RMSE_numerator + one_RMSE              
    value_denominator = value_denominator + len(pred_col)
    
MAE = MAE_numerator/value_denominator
RMSE =  math.sqrt(RMSE_numerator/value_denominator)    

print('MAE of average of six similarity pred is',MAE)
print('')
print('RMSE of average of six similarity pred is',RMSE)

100%|██████████| 15000/15000 [01:28<00:00, 173.68it/s]

MAE of average of six similarity pred is 1.07755421442

RMSE of average of six similarity pred is 1.9884731507232984


### Evaluate 2: Recall and Precision metric

In [19]:
# with open("non_active.txt", "rb") as fp:   # Unpickling
#     non_ative = pickle.load(fp)

In [24]:
# v_f = np.load('v_f.npy').item()
# index_to_user_dict = np.load('train_user_map.npy').item()
# index_to_track_id = np.load('train_track_map.npy').item()
# inv_track_id_map = np.load('inv_track_map.npy').item()
# inv_track_id_map = np.load('track_base.npy')

In [31]:
# track_base = np.load('track_base.npy').item()

In [12]:
active_user = np.array(range(15000))
active_user = np.delete(active_user,non_ative)

In [38]:
# np.save('active_user_17.npy',active_user)

In [13]:
### filter the new tracks in validation dataset which not shown in training dataset
v_ft = {u:defaultdict(int) for u in v_f.keys()}

for i in tqdm.tqdm(range(15000)):
    for k in v_f[index_to_user_dict[i]].keys():
        if k in track_base:
            v_ft[index_to_user_dict[i]][k]=v_f[index_to_user_dict[i]][k]

100%|██████████| 15000/15000 [00:00<00:00, 138446.28it/s]


In [41]:
# np.save('validation_Dict_17',v_ft)

In [308]:
recommender_cosine_U = np.load('recommender_cosine_U.npy').item()

In [309]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = recommender_cosine_U[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_cos_u = suc_rec/Tnum_val
precision_cos_u = suc_rec/num_p
print('User_based cosine similarity recall is',recall_cos_u)
print('User_based cosine similarity precision',precision_cos_u)

User_based cosine similarity recall is 0.03119329948866448
User_based cosine similarity precision 0.02906668943728119


In [310]:
recommender_Pearson_U = np.load('recommender_Pearson_U.npy').item()

In [311]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = recommender_Pearson_U[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_P_u = suc_rec/Tnum_val
precision_P_u = suc_rec/num_p
print('User_based Pearson correlation recall is',recall_P_u)
print('User_based Pearson correlation precision',precision_P_u)

User_based Pearson correlation recall is 0.013122445613327713
User_based Pearson correlation precision 0.012227819998292203


In [312]:
recommender_cosine_T = np.load('recommender_cosine_T.npy').item()

In [313]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = recommender_cosine_T[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_cos_t = suc_rec/Tnum_val
precision_cos_t = suc_rec/num_p
print('Track based cosine similarity recall is',recall_cos_t)
print('Track based cosine similarity precision is',precision_cos_t)

Track based cosine similarity recall is 0.010794860986382713
Track based cosine similarity precision is 0.01005891896507557


In [314]:
recommender_adjusted_cosine_T = np.load('recommender_adjuested_cosine_T.npy').item()

In [315]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = recommender_adjusted_cosine_T[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_adcos_t = suc_rec/Tnum_val
precision_adcos_t = suc_rec/num_p
print('Track based adjusted cosine similarity recall is',recall_adcos_t)
print('Track based adjusted cosine similarity precision is',precision_adcos_t)

Track based adjusted cosine similarity recall is 0.006322966112567124
Track based adjusted cosine similarity precision is 0.005891896507556998


In [316]:
recommender_Pearson_T = np.load('recommender_Pearson_T.npy').item()

In [317]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = recommender_Pearson_T[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_P_t = suc_rec/Tnum_val
precision_P_t = suc_rec/num_p
print('Track based Pearson correlation recall is',recall_P_t)
print('Track based Pearson correlation precision is',precision_P_t)

Track based Pearson correlation recall is 0.009713542143943698
Track based Pearson correlation precision is 0.009051319272478866


In [318]:
### combine all five recommendation sets
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    rec_set = []
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set.extend(recommender_cosine_U[i])
    rec_set.extend(recommender_Pearson_U[i])
    rec_set.extend(recommender_cosine_T[i])
    rec_set.extend(recommender_adjusted_cosine_T[i])
    rec_set.extend(recommender_Pearson_T[i])
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall = suc_rec/Tnum_val
precision = suc_rec/num_p
print('Five recommendation sets recall is',recall)
print('Five recommendation sets precision is',precision)

Five recommendation sets recall is 0.05190330443707274
Five recommendation sets precision is 0.009672957048928357


In [319]:
recommender_conten_based = np.load('recommender_content_based.npy').item()

In [320]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = recommender_conten_based[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_content = suc_rec/Tnum_val
precision_content = suc_rec/num_p
print('content-based recall is',recall_content)
print('content-based precision is',precision_content)

content-based recall is 0.00047651338819346443
content-based precision is 0.00044402698317820853


In [321]:
### combine all six recommendation sets
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    rec_set = []
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set.extend(recommender_cosine_U[i])
    rec_set.extend(recommender_Pearson_U[i])
    rec_set.extend(recommender_cosine_T[i])
    rec_set.extend(recommender_adjusted_cosine_T[i])
    rec_set.extend(recommender_Pearson_T[i])
    rec_set.extend(recommender_conten_based[i])
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall = suc_rec/Tnum_val
precision = suc_rec/num_p
print('Six recommendation sets recall is',recall)
print('Six recommendation sets precision is',precision)

Six recommendation sets recall is 0.05232483551124388
Six recommendation sets precision is 0.008126263057524265


### Filtering recommendations

In [244]:
# #read
train = pd.read_csv('train_supervised_17.csv')
train = train.drop('Unnamed: 0',axis=1)

y_label = train['ratings']
x_input = train.drop('ratings',axis=1)
x_user = x_input.drop('customer_id',axis=1)

age = pd.DataFrame(2017-train['birth_year'].values)
train = pd.concat([train,age],axis=1)
train.rename(columns={0:'age'},inplace=True)
train.drop('birth_year',axis=1,inplace=True)

train.sort_values('customer_id',inplace=True)
train = train.reset_index(drop=True)

In [245]:
user, index = np.unique(train['customer_id'].values,True)
customer = np.split(train['customer_id'].values,index[1:])

acout = np.split(train['acousticness'].values,index[1:])
rat = np.split(train['ratings'].values,index[1:])
danceability = np.split(train['danceability'].values,index[1:])
duration = np.split(train['duration_ms'].values,index[1:])
energy = np.split(train['energy'].values,index[1:])
instrumentalness = np.split(train['instrumentalness'].values,index[1:])
key = np.split(train['key'].values,index[1:])
liveness = np.split(train['liveness'].values,index[1:])
loudness = np.split(train['loudness'].values,index[1:])
mode = np.split(train['mode'].values,index[1:])
speechiness = np.split(train['speechiness'].values,index[1:])
tempo = np.split(train['tempo'].values,index[1:])
time_signature = np.split(train['time_signature'].values,index[1:])
valence = np.split(train['valence'].values,index[1:])
age = np.split(train['age'].values,index[1:])

In [246]:
cluster_X = []
for i in tqdm.tqdm(range(22000)):
    rat_ = rat[i]
    acout_ = np.sum((acout[i]*rat[i]).astype(float)/np.sum(rat[i]))
    danceability_ = np.sum((danceability[i]*rat[i]).astype(float)/np.sum(rat[i]))
    duration_ = np.sum((duration[i]*rat[i]).astype(float)/np.sum(rat[i]))
    energy_ = np.sum((energy[i]*rat[i]).astype(float)/np.sum(rat[i]))
    instrumentalness_ = np.sum((instrumentalness[i]*rat[i]).astype(float)/np.sum(rat[i]))
    key_ = np.sum((key[i]*rat[i]).astype(float)/np.sum(rat[i]))
    liveness_ = np.sum((liveness[i]*rat[i]).astype(float)/np.sum(rat[i]))
    loudness_ = np.sum((loudness[i]*rat[i]).astype(float)/np.sum(rat[i]))
    mode_ = np.sum((mode[i]*rat[i]).astype(float)/np.sum(rat[i]))
    speechiness_ = np.sum((speechiness[i]*rat[i]).astype(float)/np.sum(rat[i]))
    tempo_ = np.sum((tempo[i]*rat[i]).astype(float)/np.sum(rat[i]))
    time_signature_ = np.sum((time_signature[i]*rat[i]).astype(float)/np.sum(rat[i]))
    valence_ = np.sum((valence[i]*rat[i]).astype(float)/np.sum(rat[i]))
    age_ = np.sum((age[i]*rat[i]).astype(float)/np.sum(rat[i]))
    cluster_X.append(np.array([acout_,danceability_,duration_,energy_,instrumentalness_,key_,liveness_,loudness_,mode_,\
        speechiness_,tempo_,time_signature_,valence_,age_]))  

100%|██████████| 22000/22000 [00:04<00:00, 5320.72it/s]


In [247]:
kmeans = KMeans(n_clusters=6, random_state=0).fit(cluster_X)

In [248]:
group_1 = np.array(np.where(kmeans.labels_==0))[0]
group_2 = np.array(np.where(kmeans.labels_==1))[0]
group_3 = np.array(np.where(kmeans.labels_==2))[0]
group_4 = np.array(np.where(kmeans.labels_==3))[0]
group_5 = np.array(np.where(kmeans.labels_==4))[0]
group_6 = np.array(np.where(kmeans.labels_==5))[0]
# group_7 = np.array(np.where(kmeans.labels_==6))[0]
# group_8 = np.array(np.where(kmeans.labels_==7))[0]
# group_9 = np.array(np.where(kmeans.labels_==8))[0]
# group_10 = np.array(np.where(kmeans.labels_==9))[0]

In [322]:
raw_recommend = pd.read_csv('test_supervised.csv')
raw_recommend = raw_recommend.drop('Unnamed: 0',axis=1)
raw_recommend.sort_values('customer_id',inplace=True)
raw_recommend.reset_index(drop=True,inplace=True)

In [323]:
raw_recommend.drop_duplicates(subset=['customer_id','track_name'],inplace=True)
raw_recommend.reset_index(drop=True,inplace=True)

In [324]:
age = pd.DataFrame(2017-raw_recommend['birth_year'].values)
raw_recommend = pd.concat([raw_recommend,age],axis=1)
raw_recommend.rename(columns={0:'age'},inplace=True)
raw_recommend.drop('birth_year',axis=1,inplace=True)

In [326]:
raw_recommend['age'] = raw_recommend['age'].fillna(22.0)
# raw_recommend['age'].mode()

In [327]:
k=0
for j in tqdm.tqdm([group_1,group_2,group_3,group_4,group_5,group_6]):
    k+=1
    group_list = []
    for i in j:
        group_list.append(customer[i][0])
    if k==1:
        test_1 = raw_recommend[raw_recommend.customer_id.isin(group_list)]
    if k==2:
        test_2 = raw_recommend[raw_recommend.customer_id.isin(group_list)]
    if k==3:
        test_3 = raw_recommend[raw_recommend.customer_id.isin(group_list)]
    if k==4:
        test_4 = raw_recommend[raw_recommend.customer_id.isin(group_list)]
    if k==5:
        test_5 = raw_recommend[raw_recommend.customer_id.isin(group_list)]
    if k==6:
        test_6 = raw_recommend[raw_recommend.customer_id.isin(group_list)]
#     if k==7:
#         group_df_7 = train[train.customer_id.isin(group_list)]
#     if k==8:
#         group_df_8 = train[train.customer_id.isin(group_list)]
#     if k==9:
#         group_df_9 = train[train.customer_id.isin(group_list)]
#     if k==10:
#         group_df_10 = train[train.customer_id.isin(group_list)]

100%|██████████| 6/6 [00:00<00:00, 27.00it/s]


In [328]:
features_to_drop = ['customer_id', 'track_name','artist_name', 'album_name']
raw_recommend_train = raw_recommend.drop(features_to_drop,axis=1)
test_1 = test_1.drop(features_to_drop,axis=1)
test_2 = test_2.drop(features_to_drop,axis=1)
test_3 = test_3.drop(features_to_drop,axis=1)
test_4 = test_4.drop(features_to_drop,axis=1)
test_5 = test_5.drop(features_to_drop,axis=1)
test_6 = test_6.drop(features_to_drop,axis=1)

In [103]:
v_ft=np.load('validation_Dict.npy').item()
# active_user = np.load('active_user.npy')
# t = np.load('trainset_Dict.npy').item()
# index_to_user_dict = np.load('train_user_map.npy').item()
# index_to_track_id = np.load('train_track_map.npy').item()
# inv_track_id_map = np.load('inv_track_map.npy').item()
# track_uni_=np.load('track_feature_1.npy').item()

### XGBoost

In [329]:
filename = 'xgboost.sav'
xgboost = pickle.load(open(filename, 'rb'))

In [330]:
y_filter = xgboost.predict(raw_recommend_train)

In [331]:
pred = pd.concat([raw_recommend[['customer_id','track_name']],pd.DataFrame(y_filter).rename(columns={0:'pre_ratings'})]\
                 ,axis=1)

In [332]:
pred['pre_ratings'].describe()

count    293467.000000
mean          1.322073
std           1.131293
min           0.962987
25%           1.259450
50%           1.281012
75%           1.307150
max          56.242435
Name: pre_ratings, dtype: float64

In [333]:
user, index = np.unique(pred['customer_id'],True)
rec_track = np.split(pred['track_name'],index[1:])
pre_track = np.split(pred['pre_ratings'],index[1:])

In [334]:
keep_ = 15
filtered_rec = {}
for i in tqdm.tqdm(range(15000)):
    filtered_rec[i] = rec_track[i][pre_track[i].nlargest(keep_).index.tolist()].values.tolist()

100%|██████████| 15000/15000 [00:10<00:00, 1457.83it/s]


In [1]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = filtered_rec[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_xgb = suc_rec/Tnum_val
precision_xgb = suc_rec/num_p
print('xgboost without k-means filtered recall is',recall_xgb)
print('xgboost without k-means filtered precision is',precision_xgb)

### neural network

In [336]:
filename = 'NeuralNetwork.sav'
NN = pickle.load(open(filename, 'rb'))

In [337]:
y_NN = NN.predict(raw_recommend_train)

In [338]:
pred_NN = pd.concat([raw_recommend[['customer_id','track_name']],pd.DataFrame(y_NN).rename(columns={0:'pre_ratings'})]\
                 ,axis=1)

In [339]:
user, index = np.unique(pred_NN['customer_id'],True)
rec_track = np.split(pred_NN['track_name'],index[1:])
pre_track = np.split(pred_NN['pre_ratings'],index[1:])

In [340]:
keep_ = 15
NN_filtered_rec = {}
for i in tqdm.tqdm(range(15000)):
    NN_filtered_rec[i] = rec_track[i][pre_track[i].nlargest(keep_).index.tolist()].values.tolist()

100%|██████████| 15000/15000 [00:10<00:00, 1369.98it/s]


In [2]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = NN_filtered_rec[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_nn = suc_rec/Tnum_val
precision_nn = suc_rec/num_p
print('Neural Network without k-means filtered recall is',recall_nn)
print('Neural Network without k-means filtered precision is',precision_nn)

### random forest

In [342]:
filename = 'randaomforest_17.sav'
RFR = pickle.load(open(filename, 'rb'))

In [343]:
y_RFR = RFR.predict(raw_recommend_train)

In [344]:
pred_RFR = pd.concat([raw_recommend[['customer_id','track_name']],pd.DataFrame(y_RFR).rename(columns={0:'pre_ratings'})]\
                 ,axis=1)

In [345]:
user, index = np.unique(pred_RFR['customer_id'],True)
rec_track = np.split(pred_RFR['track_name'],index[1:])
pre_track = np.split(pred_RFR['pre_ratings'],index[1:])

In [346]:
keep_ = 15
RFR_filtered_rec = {}
for i in tqdm.tqdm(range(15000)):
    RFR_filtered_rec[i] = rec_track[i][pre_track[i].nlargest(keep_).index.tolist()].values.tolist()

100%|██████████| 15000/15000 [00:10<00:00, 1404.20it/s]


In [58]:
len(RFR_filtered_rec[0])

15

In [3]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = RFR_filtered_rec[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_rfr = suc_rec/Tnum_val
precision_rfr = suc_rec/num_p
print('Random Forest without k-means filtered recall is',recall_rfr)
print('Random Forest without k-means filtered precision is',precision_rfr)

## with K-means

### neural network

In [348]:
# neural network
filename = 'nn1.sav'
nn1 = pickle.load(open(filename, 'rb'))
filename = 'nn2.sav'
nn2 = pickle.load(open(filename, 'rb'))
filename = 'nn3.sav'
nn3 = pickle.load(open(filename, 'rb'))
filename = 'nn4.sav'
nn4 = pickle.load(open(filename, 'rb'))
filename = 'nn5.sav'
nn5 = pickle.load(open(filename, 'rb'))
filename = 'nn6.sav'
nn6 = pickle.load(open(filename, 'rb'))

In [349]:
y_nn1 = nn1.predict(test_1)
y_nn2 = nn2.predict(test_2)
y_nn3 = nn3.predict(test_3)
y_nn4 = nn4.predict(test_4)
y_nn5 = nn5.predict(test_5)
y_nn6 = nn6.predict(test_6)

In [350]:
pred_1 = pd.DataFrame(y_nn1,index=test_1.index).rename(columns={0:'pre_ratings'})
pred_2 = pd.DataFrame(y_nn2,index=test_2.index).rename(columns={0:'pre_ratings'})
pred_3 = pd.DataFrame(y_nn3,index=test_3.index).rename(columns={0:'pre_ratings'})
pred_4 = pd.DataFrame(y_nn4,index=test_4.index).rename(columns={0:'pre_ratings'})
pred_5 = pd.DataFrame(y_nn5,index=test_5.index).rename(columns={0:'pre_ratings'})
pred_6 = pd.DataFrame(y_nn6,index=test_6.index).rename(columns={0:'pre_ratings'})

In [351]:
pred = pd.concat([pred_1,pred_2,pred_3,pred_4,pred_5,pred_6])
pred = pred.reset_index().sort_values('index').reset_index()
pred.drop(['level_0','index'],axis=1,inplace=True)
pred_NN = pd.concat([raw_recommend[['customer_id','track_name']],pred],axis=1)

In [352]:
user, index = np.unique(pred_NN['customer_id'],True)
rec_track = np.split(pred_NN['track_name'],index[1:])
pre_track = np.split(pred_NN['pre_ratings'],index[1:])

In [353]:
keep_ = 15
NN_filtered_rec = {}
for i in tqdm.tqdm(range(15000)):
    NN_filtered_rec[i] = rec_track[i][pre_track[i].nlargest(keep_).index.tolist()].values.tolist()

100%|██████████| 15000/15000 [00:10<00:00, 1395.68it/s]


In [4]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = NN_filtered_rec[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_nn = suc_rec/Tnum_val
precision_nn = suc_rec/num_p
print('Neural Network with k-means filtered recall is',recall_nn)
print('Neural Network with k-means filtered precision is',precision_nn)

### xgboost

In [355]:
# xgboost
filename = 'xgboost_1.sav'
xgb_1 = pickle.load(open(filename, 'rb'))
filename = 'xgboost_2.sav'
xgb_2 = pickle.load(open(filename, 'rb'))
filename = 'xgboost_3.sav'
xgb_3 = pickle.load(open(filename, 'rb'))
filename = 'xgboost_4.sav'
xgb_4 = pickle.load(open(filename, 'rb'))
filename = 'xgboost_5.sav'
xgb_5 = pickle.load(open(filename, 'rb'))
filename = 'xgboost_6.sav'
xgb_6 = pickle.load(open(filename, 'rb'))

In [356]:
y_xgb1 = xgb_1.predict(test_1)
y_xgb2 = xgb_2.predict(test_2)
y_xgb3 = xgb_3.predict(test_3)
y_xgb4 = xgb_4.predict(test_4)
y_xgb5 = xgb_5.predict(test_5)
y_xgb6 = xgb_6.predict(test_6)

In [357]:
pred_1 = pd.DataFrame(y_xgb1,index=test_1.index).rename(columns={0:'pre_ratings'})
pred_2 = pd.DataFrame(y_xgb2,index=test_2.index).rename(columns={0:'pre_ratings'})
pred_3 = pd.DataFrame(y_xgb3,index=test_3.index).rename(columns={0:'pre_ratings'})
pred_4 = pd.DataFrame(y_xgb4,index=test_4.index).rename(columns={0:'pre_ratings'})
pred_5 = pd.DataFrame(y_xgb5,index=test_5.index).rename(columns={0:'pre_ratings'})
pred_6 = pd.DataFrame(y_xgb6,index=test_6.index).rename(columns={0:'pre_ratings'})

In [358]:
pred = pd.concat([pred_1,pred_2,pred_3,pred_4,pred_5,pred_6])
pred = pred.reset_index().sort_values('index').reset_index()
pred.drop(['level_0','index'],axis=1,inplace=True)
pred_XGB = pd.concat([raw_recommend[['customer_id','track_name']],pred],axis=1)

In [359]:
user, index = np.unique(pred_XGB['customer_id'],True)
rec_track = np.split(pred_XGB['track_name'],index[1:])
pre_track = np.split(pred_XGB['pre_ratings'],index[1:])

In [360]:
keep_ = 15
XGB_filtered_rec = {}
for i in tqdm.tqdm(range(15000)):
    XGB_filtered_rec[i] = rec_track[i][pre_track[i].nlargest(keep_).index.tolist()].values.tolist()

100%|██████████| 15000/15000 [00:10<00:00, 1424.59it/s]


In [5]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = XGB_filtered_rec[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_nn = suc_rec/Tnum_val
precision_nn = suc_rec/num_p
print('XGB with k-means filtered recall is',recall_nn)
print('XGB with k-means filtered precision is',precision_nn)

In [362]:
# randomforest
filename = 'randaomforest_17_1.sav'
rf_1 = pickle.load(open(filename, 'rb'))
filename = 'randaomforest_17_2.sav'
rf_2 = pickle.load(open(filename, 'rb'))
filename = 'randaomforest_17_3.sav'
rf_3 = pickle.load(open(filename, 'rb'))
filename = 'randaomforest_17_4.sav'
rf_4 = pickle.load(open(filename, 'rb'))
filename = 'randaomforest_17_5.sav'
rf_5 = pickle.load(open(filename, 'rb'))
filename = 'randaomforest_17_6.sav'
rf_6 = pickle.load(open(filename, 'rb'))

In [363]:
y_rf1 = rf_1.predict(test_1)
y_rf2 = rf_2.predict(test_2)
y_rf3 = rf_3.predict(test_3)
y_rf4 = rf_4.predict(test_4)
y_rf5 = rf_5.predict(test_5)
y_rf6 = rf_6.predict(test_6)

In [364]:
pred_1 = pd.DataFrame(y_rf1,index=test_1.index).rename(columns={0:'pre_ratings'})
pred_2 = pd.DataFrame(y_rf2,index=test_2.index).rename(columns={0:'pre_ratings'})
pred_3 = pd.DataFrame(y_rf3,index=test_3.index).rename(columns={0:'pre_ratings'})
pred_4 = pd.DataFrame(y_rf4,index=test_4.index).rename(columns={0:'pre_ratings'})
pred_5 = pd.DataFrame(y_rf5,index=test_5.index).rename(columns={0:'pre_ratings'})
pred_6 = pd.DataFrame(y_rf6,index=test_6.index).rename(columns={0:'pre_ratings'})

In [365]:
pred = pd.concat([pred_1,pred_2,pred_3,pred_4,pred_5,pred_6])
pred = pred.reset_index().sort_values('index').reset_index()
pred.drop(['level_0','index'],axis=1,inplace=True)
pred_RF = pd.concat([raw_recommend[['customer_id','track_name']],pred],axis=1)

In [366]:
user, index = np.unique(pred_RF['customer_id'],True)
rec_track = np.split(pred_RF['track_name'],index[1:])
pre_track = np.split(pred_RF['pre_ratings'],index[1:])

In [367]:
keep_ = 15
RF_filtered_rec = {}
for i in tqdm.tqdm(range(15000)):
    RF_filtered_rec[i] = rec_track[i][pre_track[i].nlargest(keep_).index.tolist()].values.tolist()

100%|██████████| 15000/15000 [00:11<00:00, 1354.07it/s]


In [6]:
Tnum_val = 0
suc_rec = 0
num_p = 0
for i in active_user:
    num_val = len(v_ft[index_to_user_dict[i]].keys())
    Tnum_val = Tnum_val+num_val
    rec_set = RF_filtered_rec[i]
    num_p = num_p+len(rec_set)
    for j in v_ft[index_to_user_dict[i]].keys():
        if j in rec_set:
            suc_rec+=1
            
recall_nn = suc_rec/Tnum_val
precision_nn = suc_rec/num_p
print('RF with k-means filtered recall is',recall_nn)
print('RF with k-means filtered precision is',precision_nn)